In [1]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, Var, PositiveReals, NonNegativeReals
from math import log
import pandas as pd


In [2]:
class ChemicalModel:
    def __init__(self):
        self.model = ConcreteModel()
        self.components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']
        self.parameters = Parameters()
        
        # Set params as an attribute of model
        self.model.params = self.parameters.params
        
        self.variables = Variables(self.model, self.components, self.model.params)
        self.constraints = Constraints(self.model, self.model.params)
        
    def solve(self):
        solver = SolverFactory('ipopt')
        solver.options['constr_viol_tol'] = 1e-8
        solver.options['acceptable_constr_viol_tol'] = 1e-8

        solver.solve(self.model, tee=True)

    def display_results(self):
        # Helper function to fetch the value
        def fetch_value(var):
            val = var.value
            if val is None or val < 0:
                return 0.0
            return val

        # Helper function to display and write a set of results
        def display_and_write(file, header, results):
            file.write(header + '\n')
            print(header)
            for result in results:
                file.write(result + '\n')
                print(result)

        # Specify the filename where you want to store the results
        filename = "model_results.txt"
        model = self.model 

        with open(filename, 'w') as file:
            # Write the header to the file
            file.write("Results:\n")
            print("Results:")

            # Stream S Results
            s_results = [
                f"S1: {model.params['S1']}",
                f"S2: {fetch_value(model.S2)}",
                f"S3: {fetch_value(model.S3)}",
                f"S4: {fetch_value(model.S4)}",
                f"S5: {fetch_value(model.S5)}",
                f"S6: {fetch_value(model.S6)}",
                f"S7: {fetch_value(model.S7)}"
            ]
            display_and_write(file, "\nStream S Results:", s_results)

            # d Composition Results
            components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']
            d_results = [
                f"d1[{component}]: {fetch_value(model.d1[component])}" for component in components
            ] + [
                f"d2[{component}]: {fetch_value(model.d2[component])}" for component in components
            ] + [
                f"d3[{component}]: {fetch_value(model.d3[component])}" for component in components
            ]
            display_and_write(file, "\nd Composition Results:", d_results)

            # b Composition Results
            b_results = [
                f"b1[{component}]: {fetch_value(model.b1[component])}" for component in components
            ] + [
                f"b2[{component}]: {fetch_value(model.b2[component])}" for component in components
            ] + [
                f"b3[{component}]: {fetch_value(model.b3[component])}" for component in components
            ]
            display_and_write(file, "\nb Composition Results:", b_results)

            # Generate the stream table
            stream_table_df = self.generate_stream_table()
            stream_table_str = stream_table_df.to_string()

            # Write the stream table to the file
            file.write("\nStream Table:\n")
            file.write(stream_table_str)
            file.write("\n")

        # Print to console that results are written to the file
        print(f"\nResults have been written to {filename}")


    def generate_stream_table(self):
        """Generate a table with molar flow rates of each component in each stream."""

        def fetch_value(var):
            val = var.value
            if val is None or val < 0:
                return 0.0
            return round(val, 4)  # Rounding to 4 decimal digits

        components = self.components
        data = []  # This will store rows of data which will be used to create DataFrame

        # For S1 (since it's defined in parameters)
        s1_flow = self.model.params['S1']
        s1_compositions = [self.model.params[f'z1_{component}'] for component in components]
        s1_molar_flow_rates = [s1_flow * comp for comp in s1_compositions]
        data.append(s1_molar_flow_rates)

        # For other S streams
        for i in range(2, 8):  # Start from S2 since we've already handled S1
            stream_name = "S" + str(i)
            s_flow = fetch_value(getattr(self.model, stream_name))

            if i % 2 == 0:  # Even numbered S streams
                stream_name = "d" + str(i//2)
                compositions = [fetch_value(getattr(self.model, stream_name)[component]) for component in components]
            else:  # Odd numbered S streams
                stream_name = "b" + str(i//2)
                compositions = [fetch_value(getattr(self.model, stream_name)[component]) for component in components]

            molar_flow_rates = [s_flow * comp for comp in compositions]
            data.append(molar_flow_rates)

        # Creating DataFrame
        stream_names = [f'S{i}' for i in range(1, 8)]
        df = pd.DataFrame(data, columns=components, index=stream_names)
        print(df)

        return df


In [3]:
class Constraints:
    components = ["Benzene", "Toluene", "OrthoXylene", "MetaXylene", "Ethylbenzene", "ParaXylene", "TwoMethylbutane"]

    def __init__(self, model, parameters=None):
        self.model = model
        if parameters:
            self.define_constraints(model, parameters)
        self.generate_all_constraints()

    def generate_all_constraints(self):
        for component in self.components:
            self.material_balance_rule1(component)
            self.material_balance_rule2(component)
            self.material_balance_rule3(component)

    def material_balance_rule1(self, component):
        def rule(model):
            return getattr(model, f'{component}_S1') == getattr(model, f'{component}_S2') + getattr(model, f'{component}_S3')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule1_constraint', self.model.Constraint(rule=rule))

    def material_balance_rule2(self, component):
        def rule(model):
            return getattr(model, f'{component}_S3') == getattr(model, f'{component}_S4') + getattr(model, f'{component}_S5')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule2_constraint', self.model.Constraint(rule=rule))

    def material_balance_rule3(self, component):
        def rule(model):
            return getattr(model, f'{component}_S5') == getattr(model, f'{component}_S6') + getattr(model, f'{component}_S7')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule3_constraint', self.model.Constraint(rule=rule))

    def define_constraints(self, model, parameters):
        # Map each constraint to its corresponding function
        constraints_mapping = {
            'Eq1': self.Eq1,
            'Eq2': self.Eq2,
            'Eq3': self.Eq3,
            'Eq4': self.Eq4,
            'Eq5': self.Eq5,
            'Eq6': self.Eq6,
            'Eq7': self.Eq7,
            'Eq8': self.Eq8,
            'Eq9': self.Eq9,
            'Eq10': self.Eq10,  
            'Eq11': self.Eq11,
            'Eq12': self.Eq12
        }

        components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']

        for component in components:
            # Dynamically create constraint name based on the component
            for i in range(1, 7):
                rule_name = f'{component}_comp_rule{i}'
                if rule_name in dir(self):
                    constraints_mapping[rule_name] = getattr(self, rule_name)

        # Dynamically add each constraint to the model
        for constraint_name, rule_function in constraints_mapping.items():
            setattr(model, constraint_name, Constraint(rule=rule_function))

          
    # Equations   
    def Eq1(self, model):
        return model.params['FR_S2_LK'] * model.params['S1'] * model.params['z1_Benzene'] - model.d1['Benzene'] * model.S2 == 0
    
    def Eq2(self, model):
        return (1 - model.params['FR_S2_LK']) * model.params['S1'] * model.params['z1_Benzene'] - model.b1['Benzene'] * model.S3 == 0
    
    def Eq3(self, model):
        return model.params['FR_S3_HK'] * model.params['S1'] * model.params['z1_Toluene'] == model.b1['Toluene'] * model.S3
    
    def Eq4(self, model):
        return (1 - model.params['FR_S3_HK']) * model.params['S1'] * model.params['z1_Toluene'] == model.d1['Toluene'] * model.S2
    
    def Eq5(self, model):
        return model.params['FR_S4_LK'] * model.S3 * model.b1['Toluene'] - model.d2['Toluene'] * model.S4 == 0
    
    def Eq6(self, model):
        return (1 - model.params['FR_S4_LK']) * model.S3 * model.b1['Toluene'] - model.b2['Toluene'] * model.S5 == 0
    
    def Eq7(self, model):
        return model.params['FR_S5_HK'] * model.S3 * model.b1['Ethylbenzene'] - model.b2['Ethylbenzene'] * model.S5 == 0
    
    def Eq8(self, model):
        return (1 - model.params['FR_S5_HK']) * model.S3 * model.b1['Ethylbenzene'] - model.d2['Ethylbenzene'] * model.S5 == 0
    
    def Eq9(self, model):
        return model.params['FR_S6_LK'] * model.S5 * model.b2['Ethylbenzene'] - model.d3['Ethylbenzene'] * model.S6 == 0
    
    def Eq10(self, model):
        return (1 - model.params['FR_S6_LK']) * model.S5 * model.b2['Ethylbenzene'] - model.b3['Ethylbenzene'] * model.S7 == 0
    
    def Eq11(self, model):
        return model.params['FR_S7_HK'] * model.S5 * model.b2['ParaXylene'] - model.b3['ParaXylene'] * model.S7 == 0
    
    def Eq12(self, model):
        return (1 - model.params['FR_S7_HK']) * model.S5 * model.b2['ParaXylene'] - model.d3['ParaXylene'] * model.S6 == 0


    
    # Benzene (Individual Component material balance)
    def Benzene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Benzene'] ==  model.d1['Benzene'] * model.S2 + model.b1['Benzene'] * model.S3

    def Benzene_comp_rule2(self, model):
        return model.S3 * model.b1['Benzene'] ==  model.S4 * model.d2['Benzene']
    
    def Benzene_comp_rule3(self, model):
        return model.b2['Benzene'] == 0
    
    def Benzene_comp_rule4(self, model):
        return model.d3['Benzene'] == 0
    
    def Benzene_comp_rule5(self, model):
        return model.b3['Benzene'] == 0
    
    # Toluene (Individual Component material balance)
    def Toluene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Toluene'] ==  model.d1['Toluene'] * model.S2 + model.b1['Toluene'] * model.S3
    
    def Toluene_comp_rule2(self, model):
        return model.S3 * model.b1['Toluene']  ==  model.S4 * model.d2['Toluene']  + model.S5 * model.b2['Toluene']
    
    def Toluene_comp_rule3(self, model):
        return model.b2['Toluene'] * model.S5 ==  model.S6 * model.d3['Toluene']

    def Toluene_zero_rule4(self, model):
        return model.b3['Toluene'] == 0

    
    
    # Ortho-Xylene (Individual Component material balance)
    def OrthoXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_OrthoXylene'] == model.b1['OrthoXylene'] * model.S3
    
    def OrthoXylene_comp_rule2(self, model):
        return model.d1['OrthoXylene'] == 0
    
    def OrthoXylene_comp_rule3(self, model):
        return model.b1['OrthoXylene'] * model.S3 == model.b2['OrthoXylene'] * model.S5
    
    def OrthoXylene_comp_rule4(self, model):
        return model.d2['OrthoXylene'] == 0

    def OrthoXylene_comp_rule5(self, model):
        return model.b2['OrthoXylene'] * model.S5 == model.b3['OrthoXylene'] * model.S7

    def OrthoXylene_comp_rule6(self, model):
        return model.d3['OrthoXylene'] == 0

    
    # Ethylbenzene (Individual Component material balance)
    def Ethylbenzene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Ethylbenzene'] == model.b1['Ethylbenzene'] * model.S3  
    
    def Ethylbenzene_comp_rule2(self, model):
        return model.b1['Ethylbenzene'] * model.S3  == model.d2['Ethylbenzene'] * model.S4 + model.b2['Ethylbenzene'] * model.S5

    def Ethylbenzene_comp_rule3(self, model):
        return model.b2['Ethylbenzene'] * model.S5  == model.d3['Ethylbenzene'] * model.S6 + model.b3['Ethylbenzene'] * model.S7

    def Ethylbenzene_comp_rule4(self, model):
        return model.d1['Ethylbenzene'] == 0

    
    # MetaXylene (Individual Component material balance)
    def MetaXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_MetaXylene'] == model.b1['MetaXylene'] * model.S3
    
    def MetaXylene_comp_rule2(self, model):
        return model.d1['MetaXylene'] == 0
    
    def MetaXylene_comp_rule3(self, model):
        return model.b1['MetaXylene'] * model.S3 == model.b2['MetaXylene'] * model.S5
    
    def MetaXylene_comp_rule4(self, model):
        return model.d2['MetaXylene'] == 0

    def MetaXylene_comp_rule5(self, model):
        return model.b2['MetaXylene'] * model.S5 == model.b3['MetaXylene'] * model.S7

    def MetaXylene_comp_rule6(self, model):
        return model.d3['MetaXylene'] == 0


    
    # ParaXylene (Individual Component material balance)
    def ParaXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_ParaXylene'] == model.b1['ParaXylene'] * model.S3 
    
    def ParaXylene_comp_rule2(self, model):
        return model.d1['ParaXylene'] == 0
    
    def ParaXylene_comp_rule3(self, model):
        return model.b1['ParaXylene'] * model.S3  ==  model.b2['ParaXylene'] * model.S5
    
    def ParaXylene_comp_rule4(self, model):
        return model.d2['ParaXylene'] == 0

    def ParaXylene_comp_rule5(self, model):
        return model.b2['ParaXylene'] * model.S5  == model.d3['ParaXylene'] * model.S6 + model.b3['ParaXylene'] * model.S7


    # TwoMethylbutane (Individual Component material balance)
    def TwoMethylbutane_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_TwoMethylbutane'] == model.d1['TwoMethylbutane'] * model.S2
    
    def TwoMethylbutane_comp_rule1(self, model):
        return model.b1['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule2(self, model):
        return model.d2['TwoMethylbutane'] == 0
        
    def TwoMethylbutane_comp_rule3(self, model):
        return model.b2['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule4(self, model):
        return model.d3['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule5(self, model):
        return model.b3['TwoMethylbutane'] == 0
    

In [4]:
class Parameters:
    def __init__(self):
        self.params = {
            # Molar flow rate of Stream 1 (Feed inlet)
            'S1': 4542.305, # to the power of 10**6 [kmol / hr]
            
            # Molar composition of Stream 1 (Feed inlet)
            'z1_Benzene': 0.2540,
            'z1_Toluene': 0.5840,
            'z1_OrthoXylene': 0.1374,
            'z1_MetaXylene': 0.0244,
            'z1_Ethylbenzene': 0.0,
            'z1_ParaXylene': 0.0001,
            'z1_TwoMethylbutane': 0.0,
            
            # Fractional recoveries of HK/LK
            'FR_S2_LK': 0.98,
            'FR_S3_HK': 0.95,
            
            'FR_S4_LK': 0.98,
            'FR_S5_HK': 0.95,
            
            'FR_S6_LK': 0.98,
            'FR_S7_HK': 0.95,
            
        }

In [5]:
class Variables:
    def __init__(self, model, components, parameters):
        self.define_variables(model, components, parameters)

    def define_variables(self, model, components, parameters):

        initial_value = 0.00001

        # Stream variables
        for i in range(2, 8):  # Streams from S2 to S7
            setattr(model, f'S{i}', Var(within=PositiveReals, initialize=parameters['S1']))

        # molar composition of Feed
        model.z = Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value)

        # molar compositions for Distillate and Bottom streams
        for i in range(1, 4):  # 3 sets of Distillate and Bottom streams
            setattr(model, f'd{i}', Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value))
            setattr(model, f'b{i}', Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value))



In [6]:
chemical_model = ChemicalModel()

In [7]:
chemical_model.solve()

Ipopt 3.14.12: constr_viol_tol=1e-08
acceptable_constr_viol_tol=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:      125
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:       46
                     variables with only lower bounds:        6
                variables with lower and upper bounds:       40
                     variables with only upper bounds:        0
Total number of equal

In [8]:
chemical_model.display_results()

Results:

Stream S Results:
S1: 4542.305
S2: 4086.711371513992
S3: 3986.9852262406453
S4: 4425.620706540941
S5: 3067.935274302282
S6: 3818.9554721775917
S7: 3628.0916596997495

d Composition Results:
d1[Benzene]: 0.276670030695396
d1[Toluene]: 0.03245526633579285
d1[OrthoXylene]: 0.0
d1[MetaXylene]: 0.0
d1[Ethylbenzene]: 0.0
d1[ParaXylene]: 0.0
d1[TwoMethylbutane]: 1e-05
d2[Benzene]: 0.005213937418065674
d2[Toluene]: 0.5580391003842464
d2[OrthoXylene]: 0.0
d2[MetaXylene]: 0.0
d2[Ethylbenzene]: 0.0
d2[ParaXylene]: 0.0
d2[TwoMethylbutane]: 0.0
d3[Benzene]: 0.0
d3[Toluene]: 0.013197696764780798
d3[OrthoXylene]: 0.0
d3[MetaXylene]: 0.0
d3[Ethylbenzene]: 0.0
d3[ParaXylene]: 5.947051534237947e-06
d3[TwoMethylbutane]: 0.0

b Composition Results:
b1[Benzene]: 0.005787558290442308
b1[Toluene]: 0.632074279436493
b1[OrthoXylene]: 0.15653750179267148
b1[MetaXylene]: 0.027798508324171647
b1[Ethylbenzene]: 0.0
b1[ParaXylene]: 0.00011392831280398216
b1[TwoMethylbutane]: 0.0
b2[Benzene]: 0.0
b2[Toluen

In [9]:
chemical_model.generate_stream_table()

        Benzene      Toluene  OrthoXylene  MetaXylene  Ethylbenzene  \
S1  1153.745470  2652.706120   624.112707  110.832242           0.0   
S2  1130.793044   132.818121     0.000000    0.000000           0.0   
S3    23.124514  2520.173345   623.963184  110.838189           0.0   
S4    23.013228  2469.496351     0.000000    0.000000           0.0   
S5     0.000000    50.314139   624.018040  110.752464           0.0   
S6     0.000000    50.410213     0.000000    0.000000           0.0   
S7     0.000000     0.000000   624.031772  110.656797           0.0   

    ParaXylene  TwoMethylbutane  
S1    0.454231              0.0  
S2    0.000000              0.0  
S3    0.398699              0.0  
S4    0.000000              0.0  
S5    0.306794              0.0  
S6    0.000000              0.0  
S7    0.362809              0.0  


,Benzene,Toluene,OrthoXylene,MetaXylene,Ethylbenzene,ParaXylene,TwoMethylbutane
S1,1153.745470,2652.706120,624.112707,110.832242,0.0,0.454231,0.0
S2,1130.793044,132.818121,0.000000,0.000000,0.0,0.000000,0.0
S3,23.124514,2520.173345,623.963184,110.838189,0.0,0.398699,0.0
S4,23.013228,2469.496351,0.000000,0.000000,0.0,0.000000,0.0
S5,0.000000,50.314139,624.018040,110.752464,0.0,0.306794,0.0
S6,0.000000,50.410213,0.000000,0.000000,0.0,0.000000,0.0
S7,0.000000,0.000000,624.031772,110.656797,0.0,0.362809,0.0


In 10^6 kmol / hr